#### Setting up Colab

To run this notebook on [Google's Colab](https://colab.research.google.com), you will need to perform the following steps.

Step 1. Install pyspark

Since pyspark isn't included in Colab's Python installation, you will need to install it each time you open this notebook.

In [1]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 49 kB/s 
     |████████████████████████████████| 199 kB 60.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=9b580e0eed0964129697c7d8eeb11b9b57f6bc334ea6c8742d421a89771ff67f
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark


In [2]:
!pip install composable

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 2.4 MB/s 
  Attempting uninstall: toolz
    Found existing installation: toolz 0.12.0
    Uninstalling toolz-0.12.0:
      Successfully uninstalled toolz-0.12.0


Step 2. Download and unzip the data

Next, the easiest way to access the data from the module is to download and unzip.

In [3]:
!wget https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/data.zip

--2022-10-11 14:04:43--  https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/data.zip
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/data.zip [following]
--2022-10-11 14:04:43--  https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14959102 (14M) [application/zip]
Saving to: ‘data.zip’

data.zip            100%[===================>]  14.27M  --.-KB/s    in 0.1s    

2022-10-11 14:04:44 (95.2 MB/s) - ‘data.zip’ saved [14959102/14959102]



In [4]:
!unzip data.zip

Archive:  data.zip
   creating: data/
  inflating: __MACOSX/._data         
  inflating: data/Rochester_temps_2019.xlsx  
  inflating: __MACOSX/data/._Rochester_temps_2019.xlsx  
  inflating: data/uber-raw-data-jun14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-jun14-sample.csv  
  inflating: data/heroes_information.csv  
  inflating: __MACOSX/data/._heroes_information.csv  
  inflating: data/employee.csv       
  inflating: __MACOSX/data/._employee.csv  
  inflating: data/uber-raw-data-apr14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-apr14-sample.csv  
   creating: data/baseball/
  inflating: __MACOSX/data/._baseball  
  inflating: data/uber-raw-data-may14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-may14-sample.csv  
  inflating: data/auto_sales_apr.csv  
  inflating: __MACOSX/data/._auto_sales_apr.csv  
  inflating: data/uber-raw-data-sep14-sample.csv  
  inflating: __MACOSX/data/._uber-raw-data-sep14-sample.csv  
  inflating: data/department.csv 

In [5]:
!wget https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/more_pyspark.py

--2022-10-11 14:04:44--  https://github.com/wsu-stat489/module5_intro_to_pyspark/raw/main/more_pyspark.py
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/more_pyspark.py [following]
--2022-10-11 14:04:45--  https://raw.githubusercontent.com/wsu-stat489/module5_intro_to_pyspark/main/more_pyspark.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2786 (2.7K) [text/plain]
Saving to: ‘more_pyspark.py’

more_pyspark.py     100%[===================>]   2.72K  --.-KB/s    in 0s      

2022-10-11 14:04:46 (36.2 MB/s) - ‘more_pyspark.py’ saved [2786/2786]



# Joins in `pyspark`

Performed with `df_left.join(df_right, how=type_str)`

In [6]:
from pyspark.sql import SparkSession
from more_pyspark import to_pandas
spark = SparkSession.builder.appName('Ops').getOrCreate()
dept = spark.read.csv("./data/department.csv",  header=True, inferSchema=True)
dept.collect() >> to_pandas

,DeptID,DeptName
0,31,Sales
1,33,Engineering
2,34,Clerical
3,35,Marketing


In [7]:
empl = spark.read.csv("./data/employee.csv",  header=True, inferSchema=True)
empl.collect() >> to_pandas

,LastName,DeptID
0,Rafferty,31.0
1,Jones,33.0
2,Heisenberg,33.0
3,Robinson,34.0
4,Smith,34.0
5,Williams,NaN


#### Inner join

In [8]:
(empl.join(dept, empl.DeptID == dept.DeptID, how='inner')
 .collect()) >> to_pandas

,LastName,DeptID,DeptName
0,Rafferty,31,Sales
1,Jones,33,Engineering
2,Heisenberg,33,Engineering
3,Robinson,34,Clerical
4,Smith,34,Clerical


#### Left join

In [9]:
(empl.join(dept, empl.DeptID == dept.DeptID, how='left')
 .collect()) >> to_pandas

,LastName,DeptID,DeptName
0,Rafferty,31.0,Sales
1,Jones,33.0,Engineering
2,Heisenberg,33.0,Engineering
3,Robinson,34.0,Clerical
4,Smith,34.0,Clerical
5,Williams,NaN,None


#### Right join

In [10]:
(empl.join(dept, empl.DeptID == dept.DeptID, how='right')
 .collect()) >> to_pandas

,LastName,DeptID,DeptName
0,Rafferty,31,Sales
1,Heisenberg,33,Engineering
2,Jones,33,Engineering
3,Smith,34,Clerical
4,Robinson,34,Clerical
5,None,35,Marketing


#### Outer join

In [11]:
(empl.join(dept, empl.DeptID == dept.DeptID, how='outer')
 .collect()) >> to_pandas

,LastName,DeptID,DeptName
0,Williams,NaN,None
1,Rafferty,31.0,Sales
2,Jones,33.0,Engineering
3,Heisenberg,33.0,Engineering
4,Robinson,34.0,Clerical
5,Smith,34.0,Clerical
6,None,35.0,Marketing


## Joining on multiple keys

Next, we will look at table joins that require matching multiple keys.

### Example -- Total At Bats, Hits, and Runs Allowed in 2010

To illustrate joining on multiple keys, lets

1. Compute the totals for AB, H, and R in 2010 for each team from the `Pitching` table.
2. Join on the team name and park.

This is a good example, because team information can change over the years, so we need to match both `teamID` and `yearID`.

#### Step 1. Read and process the pitching table

In [12]:
pitching = spark.read.csv("./data/baseball/core/Batting.csv", header=True, inferSchema=True)

(pitching
 .take(5)
) >> to_pandas

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NA,1,4,0,0,0,...,0,0,0,0,0,None,None,None,None,0
1,addybo01,1871,1,RC1,NA,25,118,30,32,6,...,13,8,1,4,0,None,None,None,None,0
2,allisar01,1871,1,CL1,NA,29,137,28,40,4,...,19,3,1,2,5,None,None,None,None,1
3,allisdo01,1871,1,WS3,NA,27,133,28,44,10,...,27,1,1,0,2,None,None,None,None,0
4,ansonca01,1871,1,RC1,NA,25,120,29,39,11,...,16,6,2,2,1,None,None,None,None,0


In [13]:
from pyspark.sql.functions import mean, stddev, col, sum

team_totals_2010 = (pitching
                    .select('teamID', 'yearID', 'AB', 'R', 'H')
                    .where(col('yearID') == 2010)
                    .groupBy(col('teamID'), col('yearID'))
                    .agg(sum('AB').alias('Total At Bats'),
                         sum('R').alias('Total Runs'), 
                         sum('H').alias('Total Hits'))
                   )

team_totals_2010.take(5) >> to_pandas

,teamID,yearID,Total At Bats,Total Runs,Total Hits
0,MIN,2010,5568,781,1521
1,CHA,2010,5484,752,1467
2,TOR,2010,5495,755,1364
3,FLO,2010,5531,719,1403
4,TBA,2010,5439,802,1343


#### Step 2. Read and process the teams table

In [14]:
teams = spark.read.csv("./data/baseball/core/Teams.csv", header=True, inferSchema=True)

team_name_and_park = (teams
                     .select('yearID', 'teamID', col('name').alias('Team Name'), 'park')
                     )
              
team_name_and_park.take(5) >> to_pandas

,yearID,teamID,Team Name,park
0,1871,BS1,Boston Red Stockings,South End Grounds I
1,1871,CH1,Chicago White Stockings,Union Base-Ball Grounds
2,1871,CL1,Cleveland Forest Citys,National Association Grounds
3,1871,FW1,Fort Wayne Kekiongas,Hamilton Field
4,1871,NY2,New York Mutuals,Union Grounds (Brooklyn)


#### Step 3. Perform a left-join.

Since we want to keep all rows in the totals table, and only add the team information when available, we will perform a left join on the totals table.

Notice that the second `on` argument is now a `list` of column expressions, one for each matching rule.

In [15]:
(team_totals_2010
.join(team_name_and_park,
      on = [team_totals_2010.yearID == team_name_and_park.yearID,
            team_totals_2010.teamID == team_name_and_park.teamID],
      how='left')
.take(5)
) >> to_pandas

,teamID,yearID,Total At Bats,Total Runs,Total Hits,Team Name,park
0,MIN,2010,5568,781,1521,Minnesota Twins,Target Field
1,CHA,2010,5484,752,1467,Chicago White Sox,U.S. Cellular Field
2,TOR,2010,5495,755,1364,Toronto Blue Jays,Rogers Centre
3,FLO,2010,5531,719,1403,Florida Marlins,Dolphin Stadium
4,TBA,2010,5439,802,1343,Tampa Bay Rays,Tropicana Field


## <font color="red"> Exercise 2 </font>

Determine all the players that have hit more than 100 home runs in a season.  The final table should include the players proper name, as well as the team name.  

**Hint:** You will need join the files listed below.  To get credit for this exercise, use the join `pyspark` join methods presented above.

In [ ]:
paths = ("./data/baseball/core/Batting.csv", 
         "./data/baseball/core/People.csv",
         "./data/baseball/core/Teams.csv")

In [ ]:
# Your code here